In [1]:
import pandas as pd
from openff.toolkit.topology import Molecule
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_openmm_energies

In [2]:
SMILES = "c1n(CCO)c(C(F)(F)(F))cc1CNCCl"


In [3]:
molecule = Molecule.from_smiles(SMILES)
(molecule.generate_conformers(n_conformers=20, rms_cutoff=0.1 * unit.angstrom),)
topology = molecule.to_topology()

In [5]:
sage = ForceField("two_minima_final-force-field.offxml", load_plugins = True)


In [6]:
interchange = Interchange.from_smirnoff(force_field=sage, topology=topology)


/home/evienc/miniconda3/envs/smee-descent/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: FieldInfo(annotation=NoneType, required=False, default='VirtualSites') is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [7]:
interchange.box = unit.Quantity([4, 4, 4], unit.nanometer)


In [9]:
summary = pd.DataFrame()

kj_mol = unit.kilojoule / unit.mol

for idx, conformer in enumerate(molecule.conformers):
    interchange.positions = conformer

    openmm_energies = get_openmm_energies(interchange).total_energy.m_as(kj_mol)
    # gromacs_energies = get_gromacs_energies(interchange).total_energy.m_as(kj_mol)

    summary = pd.concat(
        [
            summary,
            pd.DataFrame.from_dict(
                {
                    "Conformer No.": [idx],
                    "Interchange -> OpenMM (kJ/mol)": [round(openmm_energies, 3)],
                    # "Interchange -> GROMACS": [round(gromacs_energies, 3)],
                }
            ),
        ]
    )

In [10]:
summary.style.hide(axis="index")


Conformer No.,Interchange -> OpenMM (kJ/mol)
0,127.126000
1,223.066000
2,120.141000
3,193.710000
4,144.724000
5,142.400000
6,296.714000
7,101.070000
8,250.271000
9,224.465000
